### 导入包及数据部分

In [ ]:
import collections
from collections import defaultdict
from os.path import join

import matplotlib.pyplot as plt
import numpy as np  # 导入numpy，用于科学计算，如，矩阵运算
from sklearn.neighbors import KNeighborsClassifier  # 包装好的knn算法


def file2matrix(filename, n_features):
    fr = open(filename)
    number_of_lines = len(
        fr.readlines()
    )  # get the number of lines in the file
    # prepare matrix to return the number of features
    return_mat = np.zeros((number_of_lines, n_features))
    class_label_vector = []  # prepare labels return
    fr = open(filename)
    for index, line in enumerate(fr.readlines()):
        line = line.strip()
        list_from_line = line.split("\t")
        return_mat[index, :] = list_from_line[0:n_features]  # chose features
        class_label_vector.append(float(list_from_line[-1]))
        # classLabelVector.append(float(0))
    return return_mat, class_label_vector


### 生成KNN分类器

In [ ]:
Ls = (5, 9)
Ks = (3, 10, 24)
kNN_classifiers = ([], [])

for i, L in enumerate(Ls):

    for K in Ks:
        kNN_classifiers[i].append(KNeighborsClassifier(n_neighbors=K))

    data_x, data_y = file2matrix(join("data", "sample.dat"), L)

    # 拟合
    for kNN_classifier in kNN_classifiers[i]:
        kNN_classifier.fit(data_x, data_y)


In [ ]:
def ModulationClassTest(n_features, SNR, method, n, labels, kNN_classifier):
    accuracy = defaultdict(list)
    # 分别代表 BPSK, QPSK, 16QAM, 64QAM
    numbers = (
        defaultdict(list),
        defaultdict(list),
        defaultdict(list),
        defaultdict(list),
    )
    for snr in SNR:
        filename = (
            "test" + labels[method] + "-" + str(n) + "-" + str(snr) + ".dat"
        )
        testDataMat, _ = file2matrix(join("data", filename), n_features)
        numTestVecs = testDataMat.shape[0]
        for i in range(4):
            numbers[i][snr] = 0.0
        for i in range(numTestVecs):

            X_predict = testDataMat[i, :].reshape(1, -1)
            y_predict = kNN_classifier.predict(X_predict)

            if y_predict == 2:
                numbers[0][snr] += 1.0
            elif y_predict == 4:
                numbers[1][snr] += 1.0
            elif y_predict == 16:
                numbers[2][snr] += 1.0
            elif y_predict == 64:
                numbers[3][snr] += 1.0

        accuracy[snr] = numbers[method][snr] / numTestVecs
        print("the total correct rate on %d dB SNR is:" % snr, (accuracy[snr]))
        accuracy = collections.OrderedDict(sorted(accuracy.items()))
        for i in range(4):
            print(
                ("正确" if i == method else "") + "判断为",
                labels[i] + ":",
                numbers[i][snr],
            )
    return accuracy


Ns = (200, 500)
labels = ("BPSK", "QPSK", "16QAM", "64QAM")


def ModulationClassTests(SNR, method):
    fig, axs = plt.subplots(2, figsize=(20, 16))
    fig.suptitle("SNR vs Accuracy - " + labels[method], fontsize=32)
    x = SNR

    for i, L in enumerate(Ls):
        for N in Ns:
            for j, K in enumerate(Ks):
                label = "N = " + str(N) + ", K = " + str(K)
                print(label + ":")
                accuracy = ModulationClassTest(
                    L, SNR, method, N, labels, kNN_classifiers[i][j]
                )
                print(accuracy)

                axs[i].plot(
                    x,
                    list(accuracy.values()),
                    label=label,
                    marker="o",
                    linewidth=2.0,
                    linestyle="dashed",
                )

        axs[i].set_title("L =" + str(L), fontsize=24)
        axs[i].legend(loc="upper left", frameon=False, fontsize=18)
        axs[i].grid()

    for ax in axs.flat:
        ax.set(
            xticks=np.arange(min(x), max(x) + 1, 2.0),
            yticks=np.arange(0, 1, 0.10),
            xlabel="SNR (dB)",
            ylabel="Test accuracy",
        )

    plt.show()


### BPSK ModulationClassTest

In [ ]:
SNR = [2 * x for x in range(-2, 6)]
ModulationClassTests(SNR, 0)

### QPSK ModulationClassTest

In [ ]:
SNR = [2 * x for x in range(-2, 6)]
ModulationClassTests(SNR, 1)

### 16QAM ModulationClassTest

In [ ]:
SNR = [5 * x for x in range(0, 9)]
ModulationClassTests(SNR, 2)